# Ayudantía 06: Interfaces Gráficas II

### **Autores:** 
- Julio Huerta  ([@Julius9](https://github.com/Julius9)) 
- Felipe Vial

### Espera... **¿Otra vez threading?** Esto tiene que ser una broma.
El uso de *threading* al programar interfaces gráficas es fundamental. Solo imagina un mundo donde tu navegador de internet pudiera solo manejar una ventana o proceso a la vez: abres una ventana nueva y las otras se congelan. ¿A nadie le gustaría eso, o sí?

## Threading en PyQt

## **QThreads** 
 PyQt trae su propia implementación de *threads*, por medio de la clase llamada ``QThread``.
Se usan de manera muy parecida a los *threads* que ya conoces y **amas**, pero tienen algunas diferencias:

* A diferencia de los`threads` normales, no es posible asignarle una funcion target al `QThread`. Estos ultimos estan pensados opara ser heredados y sobreescribir su método `run()`.
* Los `QThreads` estan optimizados para su uso junto al modulo `PyQt`, por lo que funcionan mejor que los `threads` normales en este entorno.
* Los `QThreads` heredan de la clase `QObject`
* Los `QThreads` no se pueden volver `daemon`, por lo que hay tenerlo en consideracion cuando se tienen corriendo muchos subprocesos.

#### Métodos notables de QThread

 - ``isRunning``: reemplaza el método ``is_alive`` de los *threads* de Python. Permite saber si un ``QThread`` está actualmente corriendo o no.
 
 - ``start``: Idéntico al ``start`` de *threads*. Ejecuta el método ``run`` en la definición del QThread (y otras cosas importantes por detrás ;)

⚠️ **Recuerden siempre llamar a QThread.start() y no a QThread.run() directamente!!!** ⚠️

 ## QTimers
 
Un ``Qtimer`` se ejecuta periódicamente, esperando un intervalo de tiempo definido entre ciclos. La forma en que se comportan los ``QTimer`` es ideal para cualquier funcionalidad que quieras que ocurra cada cierto tiempo.

#### Métodos notables de QTimer
 - ``start`` y ``stop``: permite iniciar y parar el *timer*, respectivamente.

 - ``setInterval(ms: int)``: define que el *timer* debe emitir la señal *timeout* cada ``ms`` milisegundos.
 - ``timeout``: es la señal que llama el *timer* cuando termina el intervalo de tiempo. Puedes utilizar el método ``connect`` para conectarlo a alguna función.

 - ``isActive``: permite saber si el *timer* está actualmente corriendo (análogo a ``isRunning`` e ``is_alive``).
 - ``setSingleShot(singleShot: bool)``: permite definir si el *timer* es de tipo ``singleShot`` (entregando como parámetro ``True``). Que un *timer* sea ``singleShot`` significa que, al pasar el intervalo de tiempo,
 el timer se detendrá (es decir, no cicla indefinidamente).

## La clave del éxito: ¡Señales!
Hasta ahora, los ``QThreads`` (o *threads* en general) parecen algo que solo utilizarías si te lo piden explícitamente en la tarea... pero, en la práctica, es casi imposible implementar interfaces gráficas sin *threading*.

Una de las cosas más útiles que podemos hacer con ``QThreads`` es enviar señales entre ventanas u objetos, sin que se congelen o dejen de hacer sus respectivas funcionalidades. ¡Veamos un ejemplo!

### **Ejemplo 1** 

Intentaremos crear una ventana con:

* Un contador que muestra el numero en que va la primera cuenta
* un boton que inicia un loop de 10 repeticiones, aumenta el contador en uno en cada repetición
* Un boton contador, cuyo texto es el numero y que al ser precionado aumenta en uno

In [1]:
# Importacion de librerias para todas las celdas del ejemplo
import sys
from time import sleep
from PyQt5.QtCore import pyqtSignal, QThread, QTimer
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QPushButton

In [2]:
class VentanaSinThread(QWidget):
    actualizar_label_signal = pyqtSignal()

    def __init__(self):
        super().__init__()
        # Creamos los botones y labels necesarios para el ejemplo.
        self.label_numero = QLabel("0", self)  # Muestra el numero que ira en aumento
        self.boton_numero = QPushButton("0", self)  # Muestra el numero que sube si lo apretamos
        self.boton_loop = QPushButton("Iniciar Loop", self)  # Inicia el loop
        self.layout_principal = QVBoxLayout(self)  # Layout de la ventana principal

        self.init_gui()

In [3]:
    def init_gui(self):
        # Ordenamos las Widgets
        self.layout_principal.addWidget(self.label_numero)
        self.layout_principal.addStretch()
        self.layout_principal.addWidget(self.boton_numero)
        self.layout_principal.addWidget(self.boton_loop)
        # Conectamos las senales
        self.boton_numero.clicked.connect(self.actualizar_boton)
        self.boton_loop.clicked.connect(self.iniciar_loop)
        self.actualizar_label_signal.connect(self.actualizar_label)

        self.show()

In [4]:
    def actualizar_label(self):
        # Obtenemos el numero actual del label y lo aumentamos en 1
        numero_actual = int(self.label_numero.text())
        self.label_numero.setText(str(numero_actual + 1))

    def actualizar_boton(self):
        # Obtenemos el numero actual del boton y lo aumentamos en 1
        numero_actual = int(self.boton_numero.text())
        self.boton_numero.setText(str(numero_actual + 1))

    def iniciar_loop(self):
        # Emitimos la senal 10 veces, con 0.5 segundos de espera entre emisiones.
        for _ in range(10):
            self.actualizar_label_signal.emit()
            sleep(0.5)

In [3]:
if __name__ == '__main__':
    app = QApplication([])
    ventana = VentanaSinThread()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\juanf\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### ¿Qué pasó?
La ventana, al intentar correr todo por medio del *thread* principal, no puede procesar eventos, como apretar un botón, mientras que está corriendo el *loop*.


### **Ejemplo 2** : Solucion con `QThread`

In [ ]:
class ThreadBacan(QThread):
    def __init__(self, actualizar_label_signal, *args, **kwargs):
        # Entregar *args y **kwargs a la super clase es importante por si queremos dar algun parametro
        # inicial de los que ya ofrece la clase QThread
        super().__init__(*args, **kwargs)
        # Le entregamos una senal que queremos que el Thread emita
        self.actualizar_label_signal = actualizar_label_signal

    def run(self):
        for _ in range(10):
            self.actualizar_label_signal.emit()
            sleep(0.5)

In [ ]:
class VentanaConThread(QWidget):
    actualizar_label_signal = pyqtSignal()

    def __init__(self):
        super().__init__()
        self.label_numero = QLabel("0", self)
        self.boton_numero = QPushButton("0", self)
        self.boton_loop = QPushButton("Iniciar Loop", self)

        self.layout_principal = QVBoxLayout(self)

        # Creamos nuestro thread y le entregamos la senal para actualizar el label
        self.thread_bacan = ThreadBacan(self.actualizar_label_signal)

        self.init_gui()

In [ ]:
    def init_gui(self):
        self.layout_principal.addWidget(self.label_numero)
        self.layout_principal.addStretch()
        self.layout_principal.addWidget(self.boton_numero)
        self.layout_principal.addWidget(self.boton_loop)

        self.boton_numero.clicked.connect(self.actualizar_boton)
        self.boton_loop.clicked.connect(self.iniciar_loop)
        self.actualizar_label_signal.connect(self.actualizar_label)

        self.show()

In [ ]:
    def actualizar_label(self):
        numero_actual = int(self.label_numero.text())
        self.label_numero.setText(str(numero_actual + 1))

    def actualizar_boton(self):
        numero_actual = int(self.boton_numero.text())
        self.boton_numero.setText(str(numero_actual + 1))

    def iniciar_loop(self):
        self.thread_bacan.start()

In [ ]:
if __name__ == '__main__':
    app = QApplication([])
    ventana = VentanaConThread()
    sys.exit(app.exec_())

¡Threading puede ser muy útil!

Sin embargo, podría parecer tedioso tener que implementar un *thread* personalizado para todo lo que implique tiempo.
Es por esto que los ``QTimer`` pueden ser una herramienta muy poderosa, pues permite fácilmente enviar señales cada cierto tiempo (y nos ahorramos un poquito de código).

**Implementación con QTimer**

In [ ]:
class VentanaConTimer(QWidget):
    def __init__(self):
        super().__init__()
        self.label_numero = QLabel("0", self)
        self.boton_numero = QPushButton("0", self)
        self.boton_loop = QPushButton("Iniciar Loop", self)

        self.layout_principal = QVBoxLayout(self)
        self.timer_epico = QTimer(self)

        self.init_gui()

In [ ]:
    def init_gui(self):
        self.layout_principal.addWidget(self.label_numero)
        self.layout_principal.addStretch()
        self.layout_principal.addWidget(self.boton_numero)
        self.layout_principal.addWidget(self.boton_loop)

        self.boton_numero.clicked.connect(self.actualizar_boton)
        self.boton_loop.clicked.connect(self.iniciar_loop)

        self.show()

In [ ]:
    def actualizar_label(self):
        numero_actual = int(self.label_numero.text())
        self.label_numero.setText(str(numero_actual + 1))

        # Podemos dejar una prueba para ver si paramos el timer.
        # Idealmente, deberias implementar un timer personalizado que se preocupe de esto
        if numero_actual == 10:
            self.timer_epico.stop()

In [ ]:
    def actualizar_boton(self):
        numero_actual = int(self.boton_numero.text())
        self.boton_numero.setText(str(numero_actual + 1))

In [ ]:
    def iniciar_loop(self):
        # Los timers emiten una senal cada vez que pasa una cantidad de tiempo especificada
        # la cual puedes acceder para conectarla utilizando el atributo timeout.
        self.timer_epico.timeout.connect(self.actualizar_label)
        # Ojo: el tiempo se especifica en milisegundos!
        self.timer_epico.setInterval(1000)
        self.timer_epico.start()

In [ ]:
if __name__ == '__main__':
    app = QApplication([])
    ventana = VentanaConTimer()
    sys.exit(app.exec_())

### Ejemplo 3
Como vimos anteriormente un `QThread` no puede volverse Daemon, por lo que estos frenaran al programa a terminar. Una correcta forma de "terminarlos", es que su ejecución dependa de una variable, la cual es cambiada con el uso de una señal

In [ ]:
class ThreadDemoniaco(QThread):
    def __init__(self, padre, *args, **kwargs):
        # Entregar *args y **kwargs a la super clase es importante por si queremos dar algun parametro
        # inicial de los que ya ofrece la clase QThread
        super().__init__(*args, **kwargs)
        # Le entregamos una senal que queremos que el Thread emita
        self.ventana = padre
        self.actualizar_label_signal = self.ventana.actualizar_label_signal

    def run(self):
        while not self.ventana.terminar:
            self.actualizar_label_signal.emit()
            sleep(0.5)

In [ ]:
class VentanaConThread(QWidget):
    actualizar_label_signal = pyqtSignal()

    def __init__(self):
        super().__init__()
        self.label_numero = QLabel("0", self)
        self.boton_empezar = QPushButton("Iniciar", self)
        self.boton_terminar = QPushButton("Terminar", self)

        self.layout_principal = QVBoxLayout(self)

        # Creamos nuestro thread y le entregamos la senal para actualizar el label
        self.terminar = False
        self.thread_demoniaco = ThreadDemoniaco(self)
        self.init_gui()

    def init_gui(self):
        self.layout_principal.addWidget(self.label_numero)
        self.layout_principal.addStretch()
        self.layout_principal.addWidget(self.boton_empezar)
        self.layout_principal.addWidget(self.boton_terminar)

        self.boton_empezar.clicked.connect(self.iniciar_qthread)
        self.boton_terminar.clicked.connect(self.terminar_qthread)

        # se conecta la señal
        self.actualizar_label_signal.connect(self.actualizar_label)
        self.show()
    
    def actualizar_label(self):
        numero_actual = int(self.label_numero.text())
        self.label_numero.setText(str(numero_actual + 1))

    def iniciar_qthread(self):
        self.thread_demoniaco.start()

    def terminar_qthread(self):
        self.terminar = True

In [ ]:
if __name__ == '__main__':
    app = QApplication([])
    ventana = VentanaConThread()
    sys.exit(app.exec_())

## Main window 💻

Cuando hablamos de una ``MainWindow``, piensa simplemente en una *widget* especial 

- Trae un orden pre-definido y funcionalidades especiales.
- Facilita la construcción de aplicaciones con un orden "estándar". 

Básicamente, te permite construir rápidamente aplicaciones que ordenan sus ventanas como el *IDE* (QTdesigner) que utilizas para el ramo.

Una MainWindow se ordena de la siguiente manera:
![mainwindow](imagenes/mainwindowlayout.png)

### Elementos y métodos útiles de MainWidget

### Menús

- ``self.menubar()``: Agrega una barra de menú al MainWidget. Sobre esta podemos utilizar ``addMenu()`` para agregar un menú.

- ``addAction(accion)``: Permite agregar una acción de tipo ``QAction`` a un menú. ¿Cómo? ¿Que qué es un QAction?

### ¡QActions!

Son objetos bastante abstractos, que dentro de un menú son similares a un botón. Sin embargo, tienen un par de cosas interesantes (y útiles):
- ``setShortcut()``: Esto le asigna un *shortcut* a la acción. Introducir el shortcut tendrá el mismo efecto que hacer click sobre ella.

- ``setStatusTip(tip)``: Esto hace que cuando mantengas el ratón en *hover* sobre la acción, se muestre un mensaje de ayuda en la ``statusBar``.

- ``triggered.connect(funcion)``: Método que conecta la *activación* de la acción con otra función.

### Status Bar

Es una barra de estado :D

- ``showMessage(msg)``: Actualiza el mensaje mostrado por la statusBar (*pst... puede ser más dinámico si lo usan junto a métodos*)

### Central Widget

- ``setCentralWidget(widget)``: Asigna el cuerpo de la ventana. Pueden pensar en que MainWindow toma una ventana más pequeña, y le agrega un marco y otras cosas a su alrededor :D

## La salvación de muchos: Qt Designer 🎨🙌
Qt Designer es una herramienta de diseño que permite crear Widgets visualmente 😎

![qtdesigner](imagenes/qtdesigner-anotado.png)

## Tetris ! 🎶
* Implementaremos un juego de Tetris, para esto debemos crear la pantalla de inicio con `QtDesgner` y codigo python.

* Posteriormente analizaremos como los bloques se mueven, con la ayuda de `QThread`


